# Global Registration

In [ ]:
using Random
using DataFrames
using CoordinateTransformations, Rotations
using StaticArrays
using Gadfly
using Statistics
using NearestNeighbors
using LinearAlgebra
using GeometryBasics
using SpecialFunctions

In [ ]:
r=MersenneTwister(0xBADFEED)
npts=1000
xy = SVector.(zip(rand(r,npts),rand(r,npts)))

In [ ]:
function densitymap(data, rect, dims)
    trf(pt, dims, rect) = ceil.(Int, dims .* ((pt .- rect.origin) ./ rect.widths))
    f(x0,y0) = 0.25*(erf(x0/sqrt(2)) - erf(x1/sqrt(2)))*(erf(y0/sqrt(2)) - erf(y1/sqrt(2)))
    m = zeros(Float32, dims)
    for datum in data
        pti = trf(datum, dims, rect)
        if checkbounds(Bool, m, pti...)
            @inbounds m[pti...]+=1.0f0
        end
    end
    m
end

In [ ]:
densitymap(xy, Rect(0.25,0.25,0.5,0.5), (5,5))

In [ ]:
r = Rect(0.25,0.25,0.5,0.5)
widths = maximum(r)-minimum(r)
center = 0.5*widths+minimum(r)
fd = filter(pt->pt in r, xy)
afm0=inv(AffineMap(Angle2d{Float32}(deg2rad(31.0)), center))
fd0 = afm0.(fd)
dm0=densitymap(fd0, Rect((-0.5*sqrt(2)*widths)..., (sqrt(2)*widths)...),(10,10))
r=map(-π:π/18.0:π) do θ
    afm=inv(AffineMap(Angle2d{Float32}(θ), center))
    dm=densitymap(afm.(fd), Rect((-0.5*sqrt(2)*widths)..., (sqrt(2)*widths)...),(10,10))
    sum(dm.*dm0)
end
θopt = (-π:π/18.0:π)[findmax(r)[2]]
afmr=inv(AffineMap(Angle2d{Float32}(θopt), center))
fdr=afmr.(fd);


In [ ]:
plot(
    layer(x=map(t->t[1], fd0), y=map(t->t[2], fd0), Geom.point, Theme(default_color="red")),    
    layer(x=map(t->t[1], fdr), y=map(t->t[2], fdr), Geom.point, Theme(default_color="blue"))
)

The strategy will be to pick a rectangular region in one of the sets of points.  We will build a series of density maps of the region at over the full $2\pi$ range of rotations.  We will make a density map of the second set of points.  We will take each map in the angle series and compute how well it matches at each offset in the second map.  The best offset/angle pair will become a candidate for optimization using the Iterative Closest Point algorithm.

In [ ]:
using NearestNeighbors
kdt=KDTree(fd0)

In [ ]:
rr=knn(kdt, fdr, 1)

In [ ]:
plot(
    layer(x=map(t->t[1], fd0), y=map(t->t[2], fd0), color=map(r->r[1],rr[1]) .== 1:251, Geom.point),    
    layer(x=map(t->t[1], fdr), y=map(t->t[2], fdr), Geom.point, Theme(default_color="blue"))
)

In [ ]:
kdt=KDTree(xy)
nn2=knn(kdt,xy,2)
mean(n->n[1],nn2[2])